In [2]:
## for all ##
# path_config = PATH
# from configparser import SafeConfigParser
# parser = SafeConfigParser()
# parser.read(path_config)

from zkyhaxpy import io_tools, pd_tools, np_tools, console_tools, timer_tools, json_tools, dict_tools, log_tools
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import shutil
import re
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Any, Dict, Iterable, Iterator, List, Optional, Tuple, Union, overload

## for gis tasks##
from zkyhaxpy import gis_tools
import geopandas as gpd
import rasterio

In [21]:
dir_data = r'../data'
dir_openaq_chiangmai = os.path.join(dir_data, 'openaq', 'chiangmai_by_location')
list_files = io_tools.get_list_files_re(dir_openaq_chiangmai)
df_measurements = pd.concat([pd.read_csv(path_file) for path_file in list_files])
df_measurements['year_month'] = df_measurements['date'].str.slice(9, 16)
df_measurements['datetime'] = df_measurements['date'].str.slice(9, 25)
df_measurements['date'] = df_measurements['datetime'].str.slice(0, 10)
df_measurements['time'] = df_measurements['datetime'].str.slice(11, 17)
df_measurements

C:\Users\suras\AppData\Local\Temp\ipykernel_24036\1490146230.py:3: FutureWarning: get_list_files_re will be deprecated in the future.
  list_files = io_tools.get_list_files_re(dir_openaq_chiangmai)


Total of 561 files have been listed.


,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType,year_month,datetime,time
0,1235996,"Vieng Nua, Pai",pm25,2.191964,2023-10-31,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-31T00:00,00:00
1,1235996,"Vieng Nua, Pai",pm25,2.313988,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T23:00,23:00
2,1235996,"Vieng Nua, Pai",pm25,2.580357,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T22:00,22:00
3,1235996,"Vieng Nua, Pai",pm25,3.220238,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T21:00,21:00
4,1235996,"Vieng Nua, Pai",pm25,3.644345,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T20:00,20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,523937,Tha Kham Municipality Office,pm25,9.600000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T06:00,06:00
21,523937,Tha Kham Municipality Office,pm25,9.700000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T05:00,05:00
22,523937,Tha Kham Municipality Office,pm25,9.800000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T04:00,04:00
23,523937,Tha Kham Municipality Office,pm25,10.200000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T02:00,02:00


In [24]:
df_measurements[df_measurements['time'].between('00:00', '13:00')]

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType,year_month,datetime,time
0,1235996,"Vieng Nua, Pai",pm25,2.191964,2023-10-31,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-31T00:00,00:00
10,1235996,"Vieng Nua, Pai",pm25,1.666667,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T13:00,13:00
11,1235996,"Vieng Nua, Pai",pm25,10.683983,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T12:00,12:00
12,1235996,"Vieng Nua, Pai",pm25,16.272321,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T11:00,11:00
13,1235996,"Vieng Nua, Pai",pm25,12.257440,2023-10-30,µg/m³,"{'latitude': 19.374620435850545, 'longitude': ...",TH,NaN,False,NaN,Governmental Organization,low-cost sensor,2023-10,2023-10-30T10:00,10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,523937,Tha Kham Municipality Office,pm25,9.600000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T06:00,06:00
21,523937,Tha Kham Municipality Office,pm25,9.700000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T05:00,05:00
22,523937,Tha Kham Municipality Office,pm25,9.800000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T04:00,04:00
23,523937,Tha Kham Municipality Office,pm25,10.200000,2023-06-01,µg/m³,"{'latitude': 18.202222, 'longitude': 98.597222}",TH,NaN,False,NaN,Governmental Organization,reference grade,2023-06,2023-06-01T02:00,02:00


In [23]:
df_measurements[(df_measurements['sensorType']=='low-cost sensor')&(df_measurements['time'].between('00:00', '13:00')a)].groupby(['year_month']).agg(pm25_mean = ('value', 'mean'))

,pm25_mean
year_month,
2023-07,9.080244
2023-08,13.483098
2023-09,8.158951
2023-10,12.164699
2023-11,23.724042
2023-12,33.581755
2024-01,33.666940
2024-02,49.163122
2024-03,113.441105


In [14]:
df_measurements[df_measurements['sensorType']=='low-cost sensor'].groupby(['year_month']).agg(pm25_mean = ('value', 'mean'))a

,pm25_mean
year_month,
2023-07,9.094383
2023-08,12.851971
2023-09,8.129819
2023-10,12.092317
2023-11,23.930694
2023-12,34.353586
2024-01,35.597981
2024-02,49.615670
2024-03,115.874822
